In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine.drop(['class'], axis = 1).to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42, test_size=0.2)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state=42, test_size=0.2)

In [5]:
train_input.shape, test_input.shape, sub_input.shape, val_input.shape

((5197, 3), (1300, 3), (4157, 3), (1040, 3))

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00803185, 0.01565814, 0.00658512, 0.00698018, 0.00616479]),
 'score_time': array([0.00121331, 0.00422502, 0.00078893, 0.00089598, 0.00090122]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [8]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv= splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [17]:
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [20]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
from scipy.stats import uniform, randint

In [25]:
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 3, 1, 7, 1, 2, 9, 6, 7, 6])

In [26]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 90,  95,  93,  95, 101, 113,  99, 113, 100, 101]))

In [27]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.81500534, 0.25470654, 0.96276268, 0.6430745 , 0.48507949,
       0.44643783, 0.15159017, 0.97619535, 0.42415362, 0.36123187])

In [28]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

In [36]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f8316920b80>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f83166df5b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f831674f8e0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f8316c0a2b0>})

In [38]:
print(gs.best_params_)

{'max_depth': 37, 'min_impurity_decrease': 0.00043363254706737213, 'min_samples_leaf': 5, 'min_samples_split': 6}


In [37]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8697336566224921


In [39]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8592307692307692
